In [ ]:
import numpy as np
import hist
import matplotlib.pyplot as plt
import mplhep as hep
import pickle
import lz4.frame
from wremnants import plot_tools, scetlib_corrections,theory_tools
from wremnants import boostHistHelpers as hh
from scipy import special

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
with lz4.frame.open("/home/k/kelong/work/WRemnants/w_z_gen_dists.pkl.lz4") as minnlof:
    minnlo = pickle.load(minnlof)
minnloh = minnlo["WminusmunuPostVFP"]["output"]["nominal_gen"]*minnlo["WminusmunuPostVFP"]["dataset"]["xsec"]/minnlo["WminusmunuPostVFP"]["weight_sum"]
minnlopdfh = minnlo["ZmumuPostVFP"]["output"]["pdfNNPDF31"]*minnlo["ZmumuPostVFP"]["dataset"]["xsec"]/minnlo["ZmumuPostVFP"]["weight_sum"]
minnlohZ = minnlo["ZmumuPostVFP"]["output"]["nominal_gen"]*minnlo["ZmumuPostVFP"]["dataset"]["xsec"]/minnlo["ZmumuPostVFP"]["weight_sum"]

In [ ]:
minnloh

In [ ]:
scetlibhZ_tot = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Z/inclusive_Z.npz", add_nonsing=False)
scetlibhZ_A4 = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/inclusive_Z_pT_A4.npz", add_nonsing=False, flip_y_sign=True)

In [ ]:
scetlibhZ_A4.project("pt").plot(label="A$_{4}$")
scetlibhZ_tot.project("pt").plot(label="full")
plt.legend()

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([scetlibhZ_tot.project("y"), scetlibhZ_A4.project("y")], xlim=[-5, 5],colors=["red", "blue",], labels=["total", "A$_{4}$"], rrange=[-0.5, 0], binwnorm=1.)

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([scetlibhZ_tot.project("pt"), scetlibhZ_A4.project("pt")],colors=["red", "blue",], labels=["total", "A$_{4}$"], rrange=[-0.1, 0], binwnorm=1.)

In [ ]:
s = hist.tag.Slicer()

In [ ]:
minnlohZ.project("ptVgen").plot(label="scetlib", binwnorm=True)
scetlibhZ_tot.project("pt").plot(label="minnlo")

In [ ]:
ax = minnlohZ.axes["ptVgen"]
np.add.reduceat(minnlohZ[0,0,:,0].values(), ax.index([0, 10, 20]))

In [ ]:
np.add.reduceat(minnlohZ[0,0,:,:].values(), ax.index(scetlibhZ_tot.axes["pt"].edges))

In [ ]:
axm = minnlohZ.axes["massVgen"]
axy = minnlohZ.axes["absYVgen"]
axpt = minnlohZ.axes["ptVgen"]
minnlohZ[:,:,:,0]

In [ ]:
ipt = axpt.index(scetlibhZ_tot.axes["pt"].edges)
iy = axy.index(scetlibhZ_tot.axes["y"].edges)

In [ ]:
newh = np.add.reduceat(minnlohZ[:,:,:,0].values(), 
    ipt, axis=2
)

In [ ]:
newh = np.add.reduceat(newh, 
    (0,1), axis=1
)
np.sum(newh, axis=1, keepdims=True).shape

In [ ]:
hh.makeAbsHist(scetlibhZ_tot, "y")

In [ ]:
# TODO: Make this nd
minnlocorr = minnlohZ.project("ptVgen").copy()
scetlibcorr = minnlohZ.project("ptVgen").copy()
max_pt_corr = scetlibcorr.axes["ptVgen"].index(scetlibhZ_tot.axes["pt"].edges[-1])
scetlibcorr[:max_pt_corr] = scetlibhZ_tot[0,0,...].project("pt").view()
corr = hh.divideHists(scetlibcorr, minnlocorr, cutoff=0.01)

In [ ]:
corr.plot(label="SCETlib/MiNNLO Z ratio")
corrW.plot(label="SCETlib/MiNNLO W ratio")
plt.legend()
plt.ylim([0.8,1.2])

In [ ]:
minnlocorr[...] = minnlohZ.view(flow=True)*(1-corr[np.newaxis,np.newaxis,:,np.newaxis,])
scetlibcorr = scetlibh[0:1,...]*corr[np.newaxis,np.newaxis,np.newaxis,0:40]

In [ ]:
minnloScetlib = minnlocorr.project("ptVgen")
minnloScetlib[:40] = minnloScetlib[:40].view()+scetlibcorr.view()[0,0,0,:]

In [ ]:
minnloScetlib.plot()
minnlocorr.project("ptVgen").plot()
minnlohZ.project("ptVgen").plot()
scetlibcorr[0,0,0,:].project("pt").plot()

In [ ]:
hists = [corrh_pt] + [x .project("ptVgen") for x in [minnloh, scetlibh, minnlocorr, scetlibcorr]]
labels = [r"MiNNLO$\bigoplus$SCETlib", "MiNNLO", "SCETlib", "MiNNLO turnon", "SCETlib turnoff"]
colors = ["blue", "orange", "purple", "red", "green"]

In [ ]:
fig = plot_tools.makePlotWithRatioToRef(hists, colors=colors, labels=labels, rrange=[0.8, 1.2], binwnorm=1.)#, rlabel="x/MiNNO+Stlib")

In [ ]:
pdfPtV = minnlopdfh.project("ptVgen", "tensor_axis_0")

In [ ]:
diff = hh.addHists(pdfPtV[:, 0], -1*pdfPtV)
diff

In [ ]:
sq = hh.multiplyHists(diff, diff)

In [ ]:
ss = sq.project("ptVgen")

In [ ]:
ss[...] = np.stack((np.sqrt(ss.values()), ss.variances()), axis=-1)

In [ ]:
pdfUp = pdfPtV[:,0]+ss
pdfDown = pdfPtV[:,0]+-1*ss

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([pdfPtV[:,0], pdfUp, pdfDown], colors=["black", "red", "blue"], labels=["nominal", "up", "down"], rrange=[0.95, 1.05], binwnorm=1.)

In [ ]:
newh = hh.addHists(pdfPtV[:,0], -1*pdfPtV)

In [ ]:
newh1 = hh.addHists(pdfPtV[:,0], -1*pdfPtV[:,1])

In [ ]:
scetlibh.project("pt").plot()
minnloh.project("ptVgen").plot()

In [ ]:
scetlibh_tot = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm.npz", add_nonsing=True)
scetlibh_A4 = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm_A4.npz", add_nonsing=True, flip_y_sign=True)
scetlibh_tot_s = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm.npz", add_nonsing=False)

In [ ]:
corrh.project("ptVgen").plot()
scetlibh_tot.project("pt").plot()

In [ ]:
minnlocorr = minnloh.project("ptVgen").copy()
scetlibcorr = minnloh.project("ptVgen").copy()
scetlibcorr[...,:40] = scetlibh_tot.project("pt").view()
corrW = hh.divideHists(scetlibcorr, minnlocorr, cutoff=0.01)

In [ ]:
corr[30.j:40.j].axes["ptVgen"].centers

In [ ]:
x = corr[20.j:40.j].axes["ptVgen"].centers
params = np.polyfit(y=corr[20.j:40.j].values(), x=x, deg = 2)
poly = np.poly1d(params)
xeval = corr[30.j:40.j].axes["ptVgen"].centers

In [ ]:
corr_orig = corr.copy()
corr[30:40] = np.stack((poly(xeval), corr[30.j:40.j].variances()), axis=-1)

In [ ]:
minnloScetlib = hh.multiplyHists(minnlocorr, corrW)
minnloScetlib.plot()
matrixRadish.plot()

In [ ]:
scetlibh_A4.project("y").plot(label=r"$\sigma_{4}$")
scetlibh_tot.project("y").plot(label=r"$\sigma_{\mathrm{UL}}$")
plt.legend()

In [ ]:
scetlib_A4_coeff = hh.divideHists(scetlibh_A4, scetlibh_tot, cutoff=0.0001)
scetlib_A4_coeff_pt = hh.divideHists(scetlibh_A4.project("pt"), scetlibh_tot.project("pt"), cutoff=0.0001)

In [ ]:
scetlib_A4_coeff_pt.plot()

In [ ]:
scetlibh_A4.project("y").values()

In [ ]:
scetlibh_A4_nons.project("y").plot(label="singular")
scetlibh_A4.project("y").plot(label="nonsingular")
plt.title("A$_{4}$")
plt.legend()

In [ ]:
minnloh

In [ ]:
scetlibh_const

In [ ]:
scetlibh_A0.project("pt").plot(label="A$_{0}$")
(0.1*scetlibh_tot.project("pt")).plot(label="total")
minnloh.project("ptVgen").plot(label="minnlo")
plt.legend()

In [ ]:
scetlibhZ = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/inclusive_Z_pT.npz")

In [ ]:
scetlibhZ.project("pt").plot()
minnlohZ.project("ptVgen").plot(binwnorm=1.)

In [ ]:
scetlibh.project("pt").plot(label="W")
(100*scetlibhZ).project("pt").plot(label="Z")
plt.legend()

In [ ]:
minnloh.project("ptVgen").plot(binwnorm=1., label="MiNNLO")
scetlibh.project("pt").plot(label="SCETlib")

In [ ]:
scetlibhWpt = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-0.4-256-gbe3a9cb8/prod/scetlib_run/W/inclusive_Wm_pT.npz")

In [ ]:
scetlibhWpt[0,...].project("pt").plot(yerr=False)
minnloh.project("ptVgen").plot(binwnorm=1., label="MiNNLO")

In [ ]:
scetlibW_y_tot_s = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm_y_binMinnlo.npz")
scetlibW_y_tot_nons = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm_y_binMinnlo_nons.npz")
scetlibW_y_tot = scetlibW_y_tot_s + scetlibW_y_tot_nons

scetlibW_y_A4_s = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm_y_binMinnlo_A4.npz")
scetlibW_y_A4_nons = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm_y_binMinnlo_A4_nons.npz")
scetlibW_y_A4 = scetlibW_y_A4_s + scetlibW_y_A4_nons

#s = hist.tag.Slicer()
yax = scetlibW_y_A4.axes["y"]
scetlibW_y_A4[:,:,yax.index(0):,:] = scetlibW_y_A4[:,:,yax.index(0):,:].view()*-1

In [ ]:
scetlibW_y_tot = scetlib_corrections.readScetlibHist("/home/k/kelong/work/Generators/scetlib-cms/prod/scetlib_run/helicityTest/Wm/inclusive_Wm.npz", add_nonsing=True)

In [ ]:
scetlibW_yabs_A4 = makeYAbsHist(scetlibW_y_A4)
scetlibW_yabs_tot = makeYAbsHist(scetlibW_y_tot)

In [ ]:
yax = scetlibW_y_A4.axes["y"]
axs = scetlibW_y_A4.axes

In [ ]:
scetlib_absy_A4_coeff = hh.divideHists(scetlibW_yabs_A4.project("y"), scetlibW_yabs_tot.project("y"))

In [ ]:
scetlibW_yabs_A4.project("y").plot()
scetlibW_yabs_tot.project("y").plot()
plt.xlim([0, 4.0])

In [ ]:
hh.divideHists(scetlibW_yabs_A4.project("y"), scetlibW_yabs_tot.project("y")).plot()
plt.xlim([0,4.0])

In [ ]:
scetlibW_y_A4.project("y").plot(label="$\sigma_{4}$")
scetlibW_y_tot.project("y").plot(label="Total")
plt.legend()
plt.xlim([-4.5, 4.5])
plt.title("SCETlib structure functions")

In [ ]:
minnlohel = minnlo["WminusmunuPostVFP"]["output"]["helicity_moments_scale"][{"muRfact" : 1.j, "muFfact" : 1.j, "chargeVgen" : -1.j}]

In [ ]:
minnlohel[...,:30,4.j]

In [ ]:
minnloCoeffs = theory_tools.moments_to_angular_coeffs(minnlohel[...,:30:hist.sum,:].project("absYVgen", "helicity"))

In [ ]:
minnloCoeffs[{"helicity" : 4.j}]
#scetlib_absy_A4_coeff.plot()

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([minnloCoeffs[{"helicity" : 4.j}], scetlib_absy_A4_coeff], colors=["red", "blue"], labels=["minnlo", "scetlib"], ylabel="A$_{4}$", rrange=[0.9, 1.1], rlabel="scetlib/minnlo")

In [ ]:
blah = np.expand_dims(blah, axis=tuple(range(blah.ndim, minnlohel.ndim-blah.ndim)))

In [ ]:
minnloByHel = minnlo["WminusmunuPostVFP"]["output"]["qcdScaleByHelicity"]*minnlo["WminusmunuPostVFP"]["dataset"]["xsec"]/minnlo["WminusmunuPostVFP"]["weight_sum"]
minnloByHel

In [ ]:
import uproot

In [ ]:
f = uproot.open("/eos/user/k/kelong/HistFiles/WGen/WmToMuNu_inclusive_MATRIX_RadISH_MatchEWParams_NNPDF31.root")

In [ ]:
matrixRadish = f["wmmunu_matrix_radish/ptW_prefsr_mn"].to_hist()/1000

In [ ]:
matrixRadish.plot()
minnloh.project("ptVgen").plot()

In [ ]:
f = uproot.open("/eos/user/k/kelong/HistFiles/ZGen/ZToMuMu_MATRIX_RadISH_MatchEWParams_NNPDF31.root")

In [ ]:
f["DYm50_matrix__radish/ptZ_lhe_mm"].to_hist().plot()
minnlohZ.project("ptVgen").plot()

In [ ]:
dyturbof = np.load("/eos/user/k/kelong/HistFiles/WGen/DYTurbo/wm-13tev-cms-n3ll.npz")

In [ ]:
ptax = hist.axis.Variable(dyturbof["bins"], name="pt")
dyturboh = hist.Hist(ptax, storage=hist.storage.Weight())
dyturboh[...] = np.stack((dyturbof["hist"], dyturbof["err"]), axis=-1)

In [ ]:
dyturboMatch = matrixRadish.copy()
dyturboMatch[:40] = dyturboh.view()/1000

In [ ]:
dyturboMatch

In [ ]:
scetlibMatch = matrixRadish.copy()
scetlibMatch[:40] = scetlibh_tot.project("pt").view()

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([f["DYm50_matrix__radish/ptZ_lhe_mm"].to_hist(), minnlohZ.project("ptVgen")], colors=["blue", "green", ], labels=["MATRIX+RadISH (NNLO+N$^{3}$LL)", "MiNNLO (H$^{(2)}$ sign error)",], rrange=[0.9, 1.1], ylabel="$\sigma$/bin", xlabel="p$_{T}^{\mathrm{Z}}$ (GeV)", rlabel="x/NNLO+N$^{3}$LL", nlegcols=1)


In [ ]:
scetlibMatch

In [ ]:
fig = plot_tools.makePlotWithRatioToRef([matrixRadish, minnloScetlib, minnloh.project("ptVgen"), dyturboMatch, scetlibMatch],colors=["green", "orange", "blue", "red", "purple"], labels=["MATRIX+RadISH (NNLO+N$^{3}$LL)", "MiNNLO+SCETlib (NNLO+PS + N$^{3}$LL corr.)", "MiNNLO (NNLO+PS)", "DYTurbo (N$^{3}$LL)", "SCETlib (N$^{3}$LL)"], rrange=[0.9, 1.1], binwnorm=1., nlegcols=1, rlabel="x/Matrix+Rad", ylabel="$\sigma$/bin", xlabel="p$_{T}^{\mathrm{W}^{-}}$ (GeV)")

#fig = plot_tools.makePlotWithRatioToRef([matrixRadish, minnloh.project("ptVgen"), dyturboMatch, scetlibMatch],colors=["green", "blue", "red", "purple"], labels=["MATRIX+RadISH (NNLO+N$^{3}$LL)", "MiNNLO (NNLO+PS)", "DYTurbo (N$^{3}$LL)", "SCETlib (N$^{3}$LL)"], rrange=[0.9, 1.1], binwnorm=1., nlegcols=1, rlabel="x/Matrix+Rad", ylabel="$\sigma$/bin", xlabel="p$_{T}^{\mathrm{W}^{-}}$ (GeV)")

